In [1]:
import pandas as pd
import numpy as np
import yfinance as yf

import datetime as dt
from pandas_datareader import data as pdr
yf.pdr_override()

pd.set_option('display.max_rows',None)

This notebook calculates various technical indicators only using pandas and without using any technical analysis/indicator library.

In this notebook we calculate 
 - Moving Averages
 - Bollinger Bands
 - Relative Strength Index (RSI)
 - Moving Average Convergence Divergence (MACD)
 - Supertrend

## Moving Averages

In [2]:
ticker = input("Enter Stock Ticker : ")
print(ticker)

startyear=2012
startmonth=1
startday=1

start=dt.datetime(startyear,startmonth,startday)

end=dt.datetime.now()

stock_ma=pdr.get_data_yahoo(ticker,start,end)

Enter Stock Ticker :  RELIANCE.NS


RELIANCE.NS
[*********************100%***********************]  1 of 1 completed


In [3]:
# Simple Moving Average
stock_ma['SMA_5'] = stock_ma['Close'].rolling(5).mean()
stock_ma['SMA_10'] = stock_ma['Close'].rolling(10).mean()

In [4]:
# Exponential Moving Average
stock_ma['EMA_5'] = stock_ma['Close'].ewm(span = 5, min_periods = 4).mean()
stock_ma['EMA_10'] = stock_ma['Close'].ewm(span = 10, min_periods = 9).mean()

## Bollinger Bands

In [5]:
#ticker = input("Enter Stock Ticker : ")
print(ticker)

startyear=2012
startmonth=1
startday=1

start=dt.datetime(startyear,startmonth,startday)

end=dt.datetime.now()

stock_bbands=pdr.get_data_yahoo(ticker,start,end)

RELIANCE.NS
[*********************100%***********************]  1 of 1 completed


In [6]:
ma = stock_bbands['Close'].rolling(window = 10).mean()
stdev = stock_bbands['Close'].rolling(window = 10).std()

In [7]:
stock_bbands['middle_band'] = ma
stock_bbands['upper_band'] = ma + (2*stdev)
stock_bbands['lower_band'] = ma - (2*stdev)

## Relative Strength Index (RSI)

In [8]:
#ticker = input("Enter Stock Ticker : ")
print(ticker)

startyear=2012
startmonth=1
startday=1

start=dt.datetime(startyear,startmonth,startday)

end=dt.datetime.now()

stock_rsi=pdr.get_data_yahoo(ticker,start,end)

RELIANCE.NS
[*********************100%***********************]  1 of 1 completed


In [9]:
stock_rsi['upper_closes'] = stock_rsi['Close'].diff().clip(lower=0)
stock_rsi['lower_closes'] = (-1)*stock_rsi['Close'].diff().clip(upper=0)

In [10]:
# Calculating for 14-day RSI

stock_rsi['ma_upper_closes'] = stock_rsi['upper_closes'].ewm(com = 13, adjust = True, min_periods = 14).mean()
stock_rsi['ma_lower_closes'] = stock_rsi['lower_closes'].ewm(com = 13, adjust = True, min_periods = 14).mean()

In [11]:
stock_rsi['strength'] = stock_rsi['ma_upper_closes']/stock_rsi['ma_lower_closes']
stock_rsi['RSI'] = 100 - (100/(1+stock_rsi['strength']))

## Moving Average Convergence Divergence (MACD)

In [12]:
#ticker = input("Enter Stock Ticker : ")
print(ticker)

startyear=2012
startmonth=1
startday=1

start=dt.datetime(startyear,startmonth,startday)

end=dt.datetime.now()

stock_macd=pdr.get_data_yahoo(ticker,start,end)

RELIANCE.NS
[*********************100%***********************]  1 of 1 completed


In [13]:
stock_macd['ema_12'] = stock_macd['Close'].ewm(span=12,min_periods = 11).mean()
stock_macd['ema_26'] = stock_macd['Close'].ewm(span=26,min_periods = 25).mean()

In [14]:
stock_macd['MACD'] = stock_macd['ema_12'] - stock_macd['ema_26']
stock_macd['Signal'] = stock_macd['MACD'].ewm(span=9, adjust = False).mean()

## Supertrend

https://stackoverflow.com/questions/44935269/supertrend-code-using-pandas-python

In [15]:
#ticker = input("Enter Stock Ticker : ")
print(ticker)

startyear=2012
startmonth=1
startday=1

start=dt.datetime(startyear,startmonth,startday)

end=dt.datetime.now()

stock_st=pdr.get_data_yahoo(ticker,start,end)

RELIANCE.NS
[*********************100%***********************]  1 of 1 completed


In [16]:
stock_st = stock_st.reset_index()

In [17]:
true_range_list = []

for i in stock_st.index:
    a,b,c = (stock_st['High'][i] - stock_st['Low'][i]),(abs(stock_st['High'][i]-stock_st['Close'].shift(1)[i])),(abs(stock_st['Low'][i]-stock_st['Close'].shift(1)[i]))
    true_range = max(a,b,c)
    true_range_list.append(true_range)

In [18]:
stock_st['true_range'] = true_range_list

# Average/Smoothed True Range
stock_st['ATR'] = stock_st['true_range'].rolling(14).mean()

In [19]:
stock_st['basic_upper_band'] = round(((stock_st['High'] + stock_st['Low']) / 2) + (2 * stock_st['ATR']),2)
stock_st['basic_lower_band'] = round(((stock_st['High'] + stock_st['Low']) / 2) - (2 * stock_st['ATR']),2)

In [20]:

# FINAL UPPERBAND = IF( (Current BASICUPPERBAND < Previous FINAL UPPERBAND) or (Previous Close > Previous FINAL UPPERBAND))
#                     THEN (Current BASIC UPPERBAND) ELSE Previous FINALUPPERBAND)


for i in stock_st.index:
    if i==0:
        stock_st.loc[i,'final_upper_band']=0.00
    else:
        if (stock_st.loc[i,'basic_upper_band']<stock_st.loc[i-1,'final_upper_band'])|(stock_st.loc[i-1,"Close"]>stock_st.loc[i-1,'final_upper_band']):
            stock_st.loc[i,'final_upper_band']=stock_st.loc[i,'basic_upper_band']
        else:
            stock_st.loc[i,'final_upper_band']=stock_st.loc[i-1,'final_upper_band']

In [21]:

# FINAL LOWERBAND = IF( (Current BASIC LOWERBAND > Previous FINAL LOWERBAND) or (Previous Close < Previous FINAL LOWERBAND)) 
#                     THEN (Current BASIC LOWERBAND) ELSE Previous FINAL LOWERBAND)

for i in stock_st.index:
    if i==0:
        stock_st.loc[i,'final_lower_band']=0.00
    else:
        if (stock_st.loc[i,'basic_lower_band']>stock_st.loc[i-1,'final_lower_band'])|(stock_st.loc[i-1,"Close"]<stock_st.loc[i-1,'final_lower_band']):
            stock_st.loc[i,'final_lower_band']=stock_st.loc[i,'basic_lower_band']
        else:
            stock_st.loc[i,'final_lower_band']=stock_st.loc[i-1,'final_lower_band']

In [22]:

# SUPERTREND = IF((Previous SUPERTREND = Previous FINAL UPPERBAND) and (Current Close <= Current FINAL UPPERBAND)) THEN
#                 Current FINAL UPPERBAND
#             ELSE
#                 IF((Previous SUPERTREND = Previous FINAL UPPERBAND) and (Current Close > Current FINAL UPPERBAND)) THEN
#                     Current FINAL LOWERBAND
#                 ELSE
#                     IF((Previous SUPERTREND = Previous FINAL LOWERBAND) and (Current Close >= Current FINAL LOWERBAND)) THEN
#                         Current FINAL LOWERBAND
#                     ELSE
#                         IF((Previous SUPERTREND = Previous FINAL LOWERBAND) and (Current Close < Current FINAL LOWERBAND)) THEN
#                             Current FINAL UPPERBAND


for i in stock_st.index:
    if i==0:
        stock_st.loc[i,"ST"]=0.00
    elif (stock_st.loc[i-1,"ST"]==stock_st.loc[i-1,'final_upper_band']) & (stock_st.loc[i,"Close"]<=stock_st.loc[i,'final_upper_band']):
        stock_st.loc[i,"ST"]=stock_st.loc[i,'final_upper_band']
    elif (stock_st.loc[i-1,"ST"]==stock_st.loc[i-1,'final_upper_band'])&(stock_st.loc[i,"Close"]>stock_st.loc[i,'final_upper_band']):
        stock_st.loc[i,"ST"]=stock_st.loc[i,'final_lower_band']
    elif (stock_st.loc[i-1,"ST"]==stock_st.loc[i-1,'final_lower_band'])&(stock_st.loc[i,"Close"]>=stock_st.loc[i,'final_lower_band']):
        stock_st.loc[i,"ST"]=stock_st.loc[i,'final_lower_band']
    elif (stock_st.loc[i-1,"ST"]==stock_st.loc[i-1,'final_lower_band'])&(stock_st.loc[i,"Close"]<stock_st.loc[i,'final_lower_band']):
        stock_st.loc[i,"ST"]=stock_st.loc[i,'final_upper_band']

In [23]:
# Buy Sell Indicator
for i, row in stock_st.iterrows():
    if i==0:
        stock_st["ST_BUY_SELL"]="NA"
    elif (stock_st.loc[i,"ST"]<stock_st.loc[i,"Close"]) :
        stock_st.loc[i,"ST_BUY_SELL"]="BUY"
    else:
        stock_st.loc[i,"ST_BUY_SELL"]="SELL"

In [24]:
stock_st['final_upper_band'].count()

1